In [6]:
import numpy as np
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

ModuleNotFoundError: No module named 'torch'

In [3]:
class ConvBlock(nn.Module):
    def __init__(self, in_filters, out_filters, kernel_size=3, batchnorm=True, padding=1):
        super(ConvBlock, self).__init__()
        self.c1 = nn.Conv2d(in_filters, out_filters, kernel_size, padding=padding)
        self.bn = batchnorm
        self.b1 = nn.BatchNorm2d(out_filters)
        
    def forward(self, x):
        x = self.c1(x)
        if self.bn: 
            x = self.b1(x)
        return x

NameError: name 'nn' is not defined

In [4]:
class ResidualBlock(nn.Module):
    def __init__(self, filters, kernel_size=3, batchnorm=True, activ=F.relu, padding=1):
        '''
        filters: number of input and output filters for convolution layer
        if modifying kernel_size adjust the padding too
        '''
        super(ResidualBlock, self).__init__()
        self.activ = activ
        self.conv1 = ConvBlock(in_filters=filters, out_filters=filters, 
                               kernel_size=kernel_size, batchnorm=batchnorm, padding=padding)
        self.conv2 = ConvBlock(in_filters=filters, out_filters=filters, 
                               kernel_size=kernel_size, batchnorm=batchnorm, padding=padding)
    
    def forward(self,x):
        x_mod = self.activ(self.conv1(x))
        x_mod = self.conv2(x_mod)
        x_mod = self.activ(x_mod+x)
        return x_mod

NameError: name 'nn' is not defined

In [4]:
class PolicyHead(nn.Module):
    def __init__(self, filters=2, kernel_size=1, batchnorm=True, activ=F.relu, board_size=13, res_filters=256, padding=0):
        super(PolicyHead, self).__init__()
        self.conv = ConvBlock(in_filters=res_filters, out_filters=filters, 
                              kernel_size=kernel_size, batchnorm=batchnorm, padding=padding)
        self.activ = activ
        self.fc_infeatures = filters*board_size*board_size
        self.fc = nn.Linear(in_features=self.fc_infeatures, out_features=board_size*board_size+2)
    
    def forward(self,x):
        x = self.activ(self.conv(x))
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        # print(x)
        return torch.softmax(x,1)

In [5]:
class ValueHead(nn.Module):
    def __init__(self, filters=1, kernel_size=1, batchnorm=True, activ=F.relu, board_size=13, res_filters=256, padding=0):
        super(ValueHead, self).__init__()
        self.conv = ConvBlock(in_filters=res_filters, out_filters=filters, 
                              kernel_size=kernel_size, batchnorm=batchnorm,  padding=padding)
        self.activ = activ
        self.fc_infeatures = filters*board_size*board_size
        self.fc1 = nn.Linear(in_features=self.fc_infeatures, out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=1)
    
    def forward(self,x):
        x = self.activ(self.conv(x))
        x = x.view(x.size(0),-1)
        x = self.fc1(x)
        x = self.fc2(x)
        return torch.tanh(x)

In [1]:
class AlphaNeural(nn.Module):
    def __init__(self, res_blocks, board_size):
        super(AlphaNeural, self).__init__()
        self.res_blocks = res_blocks
        self.board_size = board_size
        self.input_stack = 17
        self.res_filters = 256
        
        self.conv1 = ConvBlock(in_filters=self.input_stack, out_filters=self.res_filters)
        for i in range(self.res_blocks):
            self.add_module("ResBlock"+str(i), ResidualBlock(filters=self.res_filters))
        self.policy_head = PolicyHead(board_size=self.board_size, res_filters=self.res_filters)
        self.value_head = ValueHead(board_size=self.board_size, res_filters=self.res_filters)
    
    def forward(self,x):
        x = torch.relu(self.conv1(x))
        # print(x.shape)
        for i in range(self.res_blocks):
            x = self._modules["ResBlock"+str(i)](x)
        # print(x.shape)

        action = self.policy_head(x)
        # print(action)
        value = self.value_head(x)
        return action, value

NameError: name 'nn' is not defined

In [7]:
model = AlphaNeural(19)

In [8]:
x = torch.rand(1,1,13,13)

In [9]:
model(x)

(tensor([[0.0031, 0.0074, 0.0049, 0.0047, 0.0043, 0.0043, 0.0051, 0.0055, 0.0073,
          0.0065, 0.0086, 0.0062, 0.0041, 0.0065, 0.0081, 0.0040, 0.0041, 0.0053,
          0.0074, 0.0023, 0.0058, 0.0063, 0.0122, 0.0070, 0.0062, 0.0088, 0.0094,
          0.0039, 0.0083, 0.0061, 0.0084, 0.0067, 0.0063, 0.0024, 0.0039, 0.0103,
          0.0060, 0.0097, 0.0061, 0.0053, 0.0053, 0.0056, 0.0084, 0.0058, 0.0049,
          0.0073, 0.0072, 0.0045, 0.0100, 0.0084, 0.0030, 0.0064, 0.0034, 0.0078,
          0.0032, 0.0045, 0.0033, 0.0042, 0.0053, 0.0048, 0.0040, 0.0062, 0.0051,
          0.0043, 0.0050, 0.0036, 0.0054, 0.0057, 0.0083, 0.0087, 0.0066, 0.0111,
          0.0055, 0.0048, 0.0042, 0.0036, 0.0057, 0.0059, 0.0060, 0.0061, 0.0061,
          0.0058, 0.0034, 0.0042, 0.0078, 0.0042, 0.0029, 0.0035, 0.0034, 0.0073,
          0.0061, 0.0052, 0.0020, 0.0037, 0.0030, 0.0064, 0.0069, 0.0057, 0.0087,
          0.0048, 0.0144, 0.0062, 0.0068, 0.0067, 0.0056, 0.0076, 0.0065, 0.0055,
          0.0035